# On the `dual` function

The  JuMP `dual` function corresponds to a slightly different notion than what it is usually understood in linear programming, as illustrated below.

In [1]:
using JuMP
using LinearAlgebra
using HiGHS

Consider the following linear program.

In [2]:
m = Model()

@variable(m, x[1:3])

@constraint(m, c1, sum(x[i] for i = 1:2) >= 3)
@constraint(m, c2, -2x[1] + 2x[2] - 4x[3] <= 5)

@constraint(m, n2, x[2] >= 0)
@constraint(m, n3, x[3] <= 0)

@objective(m, Min, 4x[1]+2x[2]+x[3])

println(m)

Min 4 x[1] + 2 x[2] + x[3]
Subject to
 c1 : x[1] + x[2] >= 3.0
 n2 : x[2] >= 0.0
 c2 : -2 x[1] + 2 x[2] - 4 x[3] <= 5.0
 n3 : x[3] <= 0.0



In [3]:
set_optimizer(m, HiGHS.Optimizer)

optimize!(m)

Presolving model
2 rows, 3 cols, 5 nonzeros
1 rows, 2 cols, 2 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-3); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  6.5000000000e+00
HiGHS run time      :          0.00


In [4]:
value.(x)

3-element Vector{Float64}:
  0.25
  2.75
 -0.0

We can use the function `dual` to directly obtain the values of the dual variables, but as explained on the page https://jump.dev/JuMP.jl/stable/manual/constraints/ the objective function is not taken into account as the function use the concept of conic duality: https://jump.dev/MathOptInterface.jl/v1.9/background/duality/#Duality

In the case of a minimization program, we obtain the desired result.

In [5]:
[dual(c1) ; dual(c2)]

2-element Vector{Float64}:
  3.0
 -0.5

JuMP also proposes the `shadow_price` function that evaluates the changes in the objective if we relax the constraint d'une unité. The term relaxation is not precisely defined, but we can see that when we have an equality greater then, the sign is the opposite to what we expect.

In [6]:
[shadow_price(c1) ; shadow_price(c2)]

2-element Vector{Float64}:
 -3.0
 -0.5

Let's explicitly form the dual program in order to validate our observations.

In [7]:
m = Model()

@variable(m, y[1:2])

@constraint(m, c1, y[1] - 2y[2] == 4)
@constraint(m, c2, y[1] + 2y[2] <= 2)
@constraint(m, c3, -4y[2] >= 1)

@constraint(m, n1, y[1] >= 0)
@constraint(m, n2, y[2] <= 0)

@objective(m, Max, 3y[1]+5y[2])

println(m)

Max 3 y[1] + 5 y[2]
Subject to
 c1 : y[1] - 2 y[2] == 4.0
 c3 : -4 y[2] >= 1.0
 n1 : y[1] >= 0.0
 c2 : y[1] + 2 y[2] <= 2.0
 n2 : y[2] <= 0.0



In [8]:
set_optimizer(m, HiGHS.Optimizer)

optimize!(m)

Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-5); columns 0(-2); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  6.5000000000e+00
HiGHS run time      :          0.00


In [9]:
value.(y)

2-element Vector{Float64}:
  3.0
 -0.5

The problem is now a maximization one, and when we call the `dual` function, we get the opposite of the primal optimal solution.

In [10]:
[ dual(c1) ; dual(c2) ; dual(c3) ]

3-element Vector{Float64}:
 -0.25
 -2.75
  0.0

Here, the shadow prices correspond to what we expect.

In [11]:
[ shadow_price(c1) ; shadow_price(c2) ; shadow_price(c3) ]

3-element Vector{Float64}:
 0.25
 2.75
 0.0

In [12]:
m = Model()

@variable(m, y[1:2])

# @constraint(m, c1, y[1] - 2y[2] == 4)
@constraint(m, c1, y[1] - 2y[2] >= 3)
@constraint(m, c1b, y[1] - 2y[2] <= 5)
@constraint(m, c2, y[1] + 2y[2] <= 2)
@constraint(m, c3, -4y[2] >= 1)

@constraint(m, n1, y[1] >= 0)
@constraint(m, n2, y[2] <= 0)

@objective(m, Max, 3y[1]+5y[2])

println(m)

set_optimizer(m, HiGHS.Optimizer)

optimize!(m)

Max 3 y[1] + 5 y[2]
Subject to
 c1 : y[1] - 2 y[2] >= 3.0
 c3 : -4 y[2] >= 1.0
 n1 : y[1] >= 0.0
 c1b : y[1] - 2 y[2] <= 5.0
 c2 : y[1] + 2 y[2] <= 2.0
 n2 : y[2] <= 0.0

Presolving model
3 rows, 2 cols, 6 nonzeros
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Presolve : Reductions: rows 2(-4); columns 2(-0); elements 4(-5)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.9999831515e+00 Ph1: 2(2); Du: 1(2.99998) 0s
          2    -6.7500000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  6.7500000000e+00
HiGHS run time      :          0.01


We can also observe this behavior from the example given at https://jump.dev/JuMP.jl/stable/manual/constraints/

In [13]:
model = Model(HiGHS.Optimizer)
@variable(model, x)
@constraint(model, con, x <= 1)
@objective(model, Min, -2x)
optimize!(model)
dual(con)

Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-1); columns 0(-1); elements 0(-1) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -2.0000000000e+00
HiGHS run time      :          0.00


-2.0

In [14]:
shadow_price(con)

-2.0

In [15]:
 @objective(model, Max, 2x)

2 x

In [16]:
optimize!(model)

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  2.0000000000e+00
HiGHS run time      :          0.00


In [17]:
dual(con)

-2.0

In [18]:
shadow_price(con)

2.0

In [19]:
model = Model(HiGHS.Optimizer)
@variable(model, x)
@constraint(model, con, -x >= 1)
@objective(model, Min, -2x)
optimize!(model)

Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-1); columns 0(-1); elements 0(-1) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  2.0000000000e+00
HiGHS run time      :          0.00


In [20]:
dual(con)

2.0

In [21]:
shadow_price(con)

-2.0

In [22]:
@objective(model, Max, 2x)
optimize!(model)

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     : -2.0000000000e+00
HiGHS run time      :          0.00


In [23]:
dual(con)

2.0

In [24]:
shadow_price(con)

2.0